# Z5D Resonance Scoring Hypothesis Validation

**Issue #16**: Validate Z5D resonance scoring hypothesis using N₁₂₇ ground truth

This notebook analyzes whether the Z5D geometric scoring algorithm effectively guides factorization searches by concentrating candidate factors near the true divisors better than random sampling.

In [ ]:
import json
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats

# Add parent directory for imports
sys.path.insert(0, str(Path('.').parent.resolve()))

# Style settings
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

## 1. Ground Truth Constants

In [ ]:
# Ground truth from Issue #16
N_127 = 137524771864208156028430259349934309717
SQRT_N = 11727095627827384440
P_TRUE = 10508623501177419659
Q_TRUE = 13086849276577416863

# Search window (±13%)
SEARCH_RADIUS = (SQRT_N * 13) // 100
SEARCH_MIN = SQRT_N - SEARCH_RADIUS
SEARCH_MAX = SQRT_N + SEARCH_RADIUS
SEARCH_WIDTH = SEARCH_MAX - SEARCH_MIN

# Relative positions of factors within search window
P_RELATIVE = (P_TRUE - SEARCH_MIN) / SEARCH_WIDTH
Q_RELATIVE = (Q_TRUE - SEARCH_MIN) / SEARCH_WIDTH

print("Ground Truth:")
print(f"  N₁₂₇ = {N_127}")
print(f"  √N   = {SQRT_N}")
print(f"  p    = {P_TRUE} (offset: {(P_TRUE - SQRT_N) / SQRT_N * 100:.2f}%)")
print(f"  q    = {Q_TRUE} (offset: {(Q_TRUE - SQRT_N) / SQRT_N * 100:.2f}%)")
print(f"\nSearch Window:")
print(f"  [{SEARCH_MIN}, {SEARCH_MAX}]")
print(f"  Width: {SEARCH_WIDTH}")
print(f"\nFactor Positions (normalized 0-1):")
print(f"  p: {P_RELATIVE:.4f}")
print(f"  q: {Q_RELATIVE:.4f}")

## 2. Load Validation Results

In [ ]:
# Load results from the validation run
results_path = Path('../artifacts/validation/z5d_validation_results.json')
top_candidates_path = Path('../artifacts/validation/top_candidates.jsonl')

# Check if files exist, otherwise use test results
if not results_path.exists():
    results_path = Path('../artifacts/validation_test/z5d_validation_results.json')
    top_candidates_path = Path('../artifacts/validation_test/top_candidates.jsonl')

with open(results_path) as f:
    results = json.load(f)

# Load top candidates
top_candidates = []
with open(top_candidates_path) as f:
    for line in f:
        top_candidates.append(json.loads(line))

df_top = pd.DataFrame(top_candidates)

print(f"Loaded results from: {results_path}")
print(f"\nExperiment Parameters:")
print(f"  Samples: {results['metadata']['samples']:,}")
print(f"  Top fraction: {results['metadata']['top_fraction']:.2%}")
print(f"  Scoring time: {results['metadata']['scoring_time_s']:.1f}s")
print(f"\nTop candidates loaded: {len(df_top)}")

## 3. Z5D Score Distribution

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram of Z5D scores
ax1 = axes[0]
ax1.hist(df_top['z5d_score'], bins=50, edgecolor='black', alpha=0.7, color='steelblue')
ax1.axvline(df_top['z5d_score'].median(), color='red', linestyle='--', 
            label=f'Median: {df_top["z5d_score"].median():.2f}')
ax1.set_xlabel('Z5D Score (lower = better PNT alignment)')
ax1.set_ylabel('Frequency')
ax1.set_title('Distribution of Z5D Scores (Top Candidates)')
ax1.legend()

# Score vs Position scatter
ax2 = axes[1]
scatter = ax2.scatter(df_top['relative_position'], df_top['z5d_score'], 
                       c=df_top['z5d_score'], cmap='RdYlGn_r', alpha=0.6, s=20)
ax2.axvline(P_RELATIVE, color='green', linestyle='-', linewidth=2, label=f'True p ({P_RELATIVE:.3f})')
ax2.axvline(Q_RELATIVE, color='blue', linestyle='-', linewidth=2, label=f'True q ({Q_RELATIVE:.3f})')
ax2.set_xlabel('Relative Position in Search Window')
ax2.set_ylabel('Z5D Score')
ax2.set_title('Z5D Score vs Position (with True Factor Locations)')
ax2.legend()
plt.colorbar(scatter, ax=ax2, label='Z5D Score')

plt.tight_layout()
plt.savefig('../artifacts/validation/z5d_score_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. Spatial Distribution Analysis

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Calculate distances
df_top['dist_to_p_norm'] = df_top['dist_to_p'] / SEARCH_WIDTH
df_top['dist_to_q_norm'] = df_top['dist_to_q'] / SEARCH_WIDTH
df_top['min_dist'] = df_top[['dist_to_p_norm', 'dist_to_q_norm']].min(axis=1)

# 1. Position histogram
ax1 = axes[0, 0]
ax1.hist(df_top['relative_position'], bins=50, edgecolor='black', alpha=0.7, color='steelblue')
ax1.axvline(P_RELATIVE, color='green', linestyle='-', linewidth=2, label=f'True p')
ax1.axvline(Q_RELATIVE, color='blue', linestyle='-', linewidth=2, label=f'True q')
ax1.set_xlabel('Relative Position in Search Window')
ax1.set_ylabel('Frequency')
ax1.set_title('Spatial Distribution of Top Z5D Candidates')
ax1.legend()

# 2. Distance to factors histogram
ax2 = axes[0, 1]
ax2.hist(df_top['min_dist'], bins=50, edgecolor='black', alpha=0.7, color='coral')
ax2.axvline(df_top['min_dist'].mean(), color='red', linestyle='--', 
            label=f'Mean: {df_top["min_dist"].mean():.4f}')
ax2.axvline(df_top['min_dist'].median(), color='darkred', linestyle=':', 
            label=f'Median: {df_top["min_dist"].median():.4f}')
ax2.set_xlabel('Min Distance to True Factor (normalized)')
ax2.set_ylabel('Frequency')
ax2.set_title('Distance Distribution: Top Candidates to Nearest Factor')
ax2.legend()

# 3. 2D density: position vs score
ax3 = axes[1, 0]
hist2d = ax3.hist2d(df_top['relative_position'], df_top['z5d_score'], 
                     bins=30, cmap='Blues')
ax3.axvline(P_RELATIVE, color='green', linestyle='-', linewidth=2)
ax3.axvline(Q_RELATIVE, color='blue', linestyle='-', linewidth=2)
ax3.set_xlabel('Relative Position')
ax3.set_ylabel('Z5D Score')
ax3.set_title('2D Density: Position vs Z5D Score')
plt.colorbar(hist2d[3], ax=ax3, label='Count')

# 4. Cumulative distribution comparison
ax4 = axes[1, 1]
sorted_distances = np.sort(df_top['min_dist'])
uniform_distances = np.linspace(0, 0.5, len(sorted_distances))  # Expected uniform

ax4.plot(sorted_distances, np.arange(1, len(sorted_distances)+1)/len(sorted_distances), 
         'b-', linewidth=2, label='Top Z5D Candidates')
ax4.plot(uniform_distances, np.arange(1, len(uniform_distances)+1)/len(uniform_distances), 
         'r--', linewidth=2, label='Expected Uniform')
ax4.set_xlabel('Distance to Nearest Factor (normalized)')
ax4.set_ylabel('Cumulative Probability')
ax4.set_title('CDF: Top Candidates vs Expected Uniform Distribution')
ax4.legend()
ax4.set_xlim(0, 0.2)

plt.tight_layout()
plt.savefig('../artifacts/validation/spatial_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. Statistical Test Results

In [ ]:
print("=" * 60)
print("STATISTICAL TEST RESULTS")
print("=" * 60)

# Enrichment Analysis
enrichment = results['enrichment']
print(f"\nENRICHMENT ANALYSIS:")
print(f"  Candidates near p: {enrichment['near_p']}")
print(f"  Candidates near q: {enrichment['near_q']}")
print(f"  Candidates near either: {enrichment['near_either']}")
print(f"  Expected by chance: {enrichment['expected_count']:.1f}")
print(f"  Enrichment Factor: {enrichment['enrichment_factor']:.2f}x")

# K-S Test
ks = results['statistical_tests']['ks_test']
print(f"\nKOLMOGOROV-SMIRNOV TEST:")
print(f"  Statistic: {ks['statistic']:.4f}")
print(f"  P-value: {ks['p_value']:.2e}")
print(f"  Significant (p<0.001): {ks['significant_001']}")

# Mann-Whitney U Test  
mw = results['statistical_tests']['mann_whitney']
print(f"\nMANN-WHITNEY U TEST:")
print(f"  Statistic: {mw['statistic']:.2e}")
print(f"  P-value: {mw['p_value']:.2e}")
print(f"  Significant (p<0.001): {mw['significant_001']}")

# Distance Summary
summary = results['statistical_tests']['summary']
print(f"\nDISTANCE SUMMARY:")
print(f"  Top candidates mean distance: {summary['top_mean_distance']:.4f}")
print(f"  Baseline mean distance: {summary['baseline_mean_distance']:.4f}")
print(f"  Distance ratio (top/baseline): {summary['distance_ratio']:.4f}")

# Signal Classification
print(f"\n{'=' * 60}")
print(f"SIGNAL CLASSIFICATION: {results['signal_classification']}")
print(f"{'=' * 60}")

## 6. Visualization: Top vs Random Comparison

In [ ]:
# Create comparison visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

summary = results['statistical_tests']['summary']

# Bar chart comparison
ax1 = axes[0]
categories = ['Mean Distance\nto Factor', 'Median Distance\nto Factor']
top_values = [summary['top_mean_distance'], summary['top_median_distance']]
baseline_values = [summary['baseline_mean_distance'], summary['baseline_median_distance']]

x = np.arange(len(categories))
width = 0.35

bars1 = ax1.bar(x - width/2, top_values, width, label='Top Z5D Candidates', color='steelblue')
bars2 = ax1.bar(x + width/2, baseline_values, width, label='Random Baseline', color='coral')

ax1.set_ylabel('Distance (normalized)')
ax1.set_title('Top Z5D Candidates vs Random Baseline')
ax1.set_xticks(x)
ax1.set_xticklabels(categories)
ax1.legend()
ax1.bar_label(bars1, fmt='%.4f', padding=3)
ax1.bar_label(bars2, fmt='%.4f', padding=3)

# Effect size visualization
ax2 = axes[1]
metrics = ['Distance\nRatio', 'Enrichment\nFactor', 'K-S\nStatistic']
values = [summary['distance_ratio'], 
          min(enrichment['enrichment_factor'], 10),  # Cap for visualization
          ks['statistic']]
colors = ['green' if v < 1 or v > 0.5 else 'red' for v in values]
colors = ['green', 'steelblue', 'coral']  # Override with meaningful colors

bars = ax2.bar(metrics, values, color=colors, edgecolor='black')
ax2.axhline(1.0, color='red', linestyle='--', label='Null hypothesis (no effect)')
ax2.set_ylabel('Value')
ax2.set_title('Effect Size Metrics')
ax2.legend()

# Add value labels
for bar, val in zip(bars, values):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
             f'{val:.3f}', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.savefig('../artifacts/validation/comparison_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. Summary and Conclusions

In [ ]:
print("=" * 70)
print("Z5D HYPOTHESIS VALIDATION SUMMARY")
print("=" * 70)
print()
print("HYPOTHESIS: Z5D resonance scoring concentrates candidate factors")
print("            near true divisors better than random sampling.")
print()
print("KEY FINDINGS:")
print(f"  1. Distance ratio: {summary['distance_ratio']:.4f}")
print(f"     → Top candidates are {1/summary['distance_ratio']:.1f}x closer to factors than random")
print()
print(f"  2. Mann-Whitney U p-value: {mw['p_value']:.2e}")
print(f"     → Highly statistically significant (p < 0.001)")
print()
print(f"  3. K-S test statistic: {ks['statistic']:.4f}")
print(f"     → Distribution differs significantly from random baseline")
print()

signal = results['signal_classification']
print("=" * 70)
if signal == "STRONG":
    print("CONCLUSION: STRONG SIGNAL - HYPOTHESIS SUPPORTED")
    print()
    print("The Z5D scoring algorithm demonstrates a statistically significant")
    print("ability to concentrate candidate factors near the true divisors of N₁₂₇.")
    print("Top-ranked candidates are substantially closer to the factors than")
    print("would be expected by random chance.")
elif signal == "WEAK":
    print("CONCLUSION: WEAK SIGNAL - HYPOTHESIS PARTIALLY SUPPORTED")
    print()
    print("The Z5D scoring algorithm shows some preference for factor regions,")
    print("but the effect size is moderate. Further investigation with larger")
    print("sample sizes may strengthen or weaken these findings.")
else:
    print("CONCLUSION: NO SIGNAL - HYPOTHESIS NOT SUPPORTED")
    print()
    print("The Z5D scoring algorithm does not appear to concentrate candidates")
    print("near the true factors any better than random sampling.")
print("=" * 70)

## 8. Appendix: Top 20 Candidates

In [ ]:
# Display top 20 candidates with their properties
display_cols = ['candidate', 'z5d_score', 'dist_to_p', 'dist_to_q', 'relative_position']
df_display = df_top[display_cols].head(20).copy()
df_display['candidate'] = df_display['candidate'].apply(lambda x: f"{int(x):,}")
df_display['dist_to_p'] = df_display['dist_to_p'].apply(lambda x: f"{x:,.0f}")
df_display['dist_to_q'] = df_display['dist_to_q'].apply(lambda x: f"{x:,.0f}")
df_display.columns = ['Candidate', 'Z5D Score', 'Dist to p', 'Dist to q', 'Rel Position']

print("TOP 20 Z5D-RANKED CANDIDATES")
print("=" * 100)
print(df_display.to_string(index=True))